In [ ]:
# Bibliotecas
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import model_from_json

In [ ]:
# Carregando Dados
dataset = pd.read_csv("../Databases/iris.csv")

#  PRE-PROCESSMENTO 
# Separando em Features e Labels;
data = dataset.iloc[:,:-1]
labels = dataset.iloc[:,-1]

# Deletando dataframe;
del dataset

# Transformando dados categoricos em numericos;
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

# Transformando dados numericos em Dummy;
labels = to_categorical(labels)

# Dividindo entre dados de treino e de teste;
X_train, X_test, y_train, y_test = train_test_split(data, labels)

In [ ]:
# Criando rede neural;

# - Função de criação de rede neural;
def createNetwork():

    model = Sequential()
    # Adicionando camadas;
    model.add(Dense(units=4, activation="relu", input_dim=4))
    model.add(Dropout(0.2))
    model.add(Dense(units=4, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(units=3, activation="softmax"))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

    return model

In [ ]:
# Construindo Modelo;
model = createNetwork()

# Treinando modelo;
model.fit(x=X_train,y=y_train, batch_size=10, epochs=1000)


In [10]:
# Avaliação do modelo;
result = model.evaluate(x=X_test, # Dados de teste;
                        y=y_test) # Labels de teste;

# Fazendo previsão usando da base de teste;
pred = model.predict(x=X_test)   # Dados para serem previstos;
pred = (pred > 0.5)               # Fará a conversão para true se o valor for maior que 0.5 e false se for menor;

# Transformando do formato Dummy para numerioco, para realizar a avaliação;
y_test_2 = [np.argmax(t) for t in y_test]
pred_2 = [np.argmax(t) for t in y_test]

# Matriz de  confusão
matriz = confusion_matrix(pred_2,y_test_2)
pd.DataFrame(matriz)

38/38 [==============================] - 0s 157us/sample - loss: 0.0635 - categorical_accuracy: 1.0000


,0,1,2
0,13,0,0
1,0,11,0
2,0,0,14


In [ ]:
model_json = model.to_json()

with open('classificador_iris.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('classificador_iris.h5')

In [ ]:
# Lendo .json que contem a estrutura do modelo;
file_model = open('classificador_iris.json', 'r')
estrutura_rede = file_model.read()
file_model.close()

# Carregando modelo;
classificador = model_from_json(estrutura_rede)
# Carregando pesos do modelo;
classificador.load_weights('classificador_iris.h5')

In [11]:
# Fazendo classificação de um unico registro;
result = classificador.predict(X_test.iloc[0:1:].values)
# Fará a conversão para true se o valor for maior que 0.5 e false se for menor;
result = (result > 0.5)               

# Transformando do formato Dummy para numerioco, para realizar a avaliação;
y_result_2 = [np.argmax(t) for t in y_test]
result_2 = [np.argmax(t) for t in y_test]

In [12]:
result

array([[ True, False, False]])